In [11]:
# Imports
import pandas as pd

from KNNAlgo import*
from helpersKNNmeans import*
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# Load dataset and samples into a pandas data frame
DATA_TRAIN_PATH = 'data/data_train.csv'
data_np = load_data(DATA_TRAIN_PATH)


DATA_TEST_PATH = 'data/sampleSubmission.csv'
samples = load_data(DATA_TEST_PATH)

# Split the data set into two sets:
#      -Training set : 80% of the data set
#      - Test set : 20% of the data set
#
# A seed is added to reproduce our results

train_data_np, test_data_np = train_test_split(data_np[['user_id', 'movie_id', 'rating']], 
                                               test_size = 0.2 ,random_state=1)

## Predictions for the submission

In [10]:
def predictionKNN (data_np, samples, test_purpose = False):
    """Make predictions for the k-Nearest Neighbours and return a .csv file.
    The function contain the following models:
        - k-Nearest Neighbours basic user based and item based
        - k-Nearest Neighbours with means user based and item based
        - k-Nearest Neighbours with z score user based and item based
        
            test_purpose : boolean in case the samples variable and data_np are 
            there for a test purpose
            samples : set where the predictions are made. This set is returned into the csv file 
            data_np : training set to train the model
            
     """
    
    computeKNNBasicUser(data_np, samples, test_purpose)
    computeKNNBasicMovie(data_np, samples, test_purpose)
    
    computeKNNMeansUser(data_np, samples, test_purpose)
    computeKNNMeansMovie(data_np, samples, test_purpose)
    
    computeKNNZScoreUser(data_np, samples, test_purpose)
    computeKNNZScoreMovie(data_np, samples, test_purpose)


predictionKNN(train_data_np, test_data_np, True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


KeyboardInterrupt: 